In [ ]:
import os
import numpy as np
from skimage import io, transform
import pickle
import torch
from behavenet.models import AE
from test_tube import HyperOptArgumentParser
from data.data_generator import ConcatSessionsGenerator

In [1]:
# specify decoder directory
ids = {
    'lab': 'musall',
    'expt': 'vistrained',
    'animal': 'mSM30',
    'session': '10-Oct-2017'}
data_dir = '/home/mattw/data/'
tt_expt = 'decoder_grid_search'    